In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from model.rcv1 import DataConfig
from model.reassignment import DocIdReassignment
from model.clustering import KMeansClustering
from model.d_gap import DGapComputation, DGapComputationReassigned, DGapInference
from model.rcv1 import RCV1Loader

In [2]:
config = DataConfig(name="rcv1-100", n_cluster=100)

In [ ]:

loader = RCV1Loader()
collection = loader.load()

In [ ]:
print("  -  Computing d-gap...")
dgap = DGapComputation(collection=collection, data_name=config.name)
dgap.compute_d_gaps()
dgap.save_d_gaps()

In [ ]:
print("  -  Evaluating clustering...")
kmeans = KMeansClustering(collection=collection, data_name=config.name, k=config.n_cluster)
kmeans.fit()
kmeans.save_labeling()

In [39]:
print("  -  Computing centroids...")
collection_clusters = kmeans.clusters
collection_clusters.compute_centroids(try_load=False)
collection_clusters.save_centroids()

  -  Computing centroids...


100%|██████████| 100/100 [00:00<00:00, 264.66it/s]


In [21]:
print("  -  Reassign doc-id...")
reassignment_computation = DocIdReassignment(
    cluster=collection_clusters,
    data_name=config.name
)

  -  Reassign doc-id...


In [ ]:
reassignment_computation.save_order()
collection_reassigned = reassignment_computation.reassign_doc_id()

print("  -  Computing d-gap after reassignment...")
dgap_reass = DGapComputationReassigned(collection=collection_reassigned, data_name=config.name)
dgap_reass.compute_d_gaps()
dgap_reass.save_d_gaps()

print("  -  Performing inference...")
inference = DGapInference(d_gap_original=dgap, d_gap_reassigned=dgap_reass, data_name=config.name)
inference.plot_avg_d_gap()
inference.avg_compression